# Sentiment Analysis

In [79]:
#import dependancies
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import hvplot.pandas
from datetime import timedelta , datetime
from config import api_key

In [2]:
#Read in csv files

nyse_price_csv = Path("Resources/nyse_prices-split-adjusted.csv")

nyse_sec_csv = Path("Resources/nyse_securities.csv")

spx_csv = Path("Resources/SPX.csv")

#create df's
nyse_price_df = pd.read_csv(nyse_price_csv)
nyse_sec_df = pd.read_csv(nyse_sec_csv)
spx_df = pd.read_csv(spx_csv)


In [3]:
#Preview NYSE price data
nyse_price_df.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [4]:
#Preview NYSE securities data
nyse_sec_df.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [5]:
#Preview NYSE SP500 data
spx_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


# Data Cleaning

In [6]:
# Lets look for null values in the nyse price dataset
nyse_price_df.isna().sum()
#Lets change symbol column to Ticker Symbol for merging
nyse_price_df["Ticker symbol"] = nyse_price_df["symbol"]
# drop old symbol column
nyse_price_df= nyse_price_df.drop(["symbol"],axis = 1)
#set index to ticker symbol
#nyse_price_df = nyse_price_df.set_index(nyse_price_df["Ticker symbol"])
nyse_price_df.head()

,date,open,close,low,high,volume,Ticker symbol
0,2016-01-05,123.430000,125.839996,122.309998,126.250000,2163600.0,WLTW
1,2016-01-06,125.239998,119.980003,119.940002,125.540001,2386400.0,WLTW
2,2016-01-07,116.379997,114.949997,114.930000,119.739998,2489500.0,WLTW
3,2016-01-08,115.480003,116.620003,113.500000,117.440002,2006300.0,WLTW
4,2016-01-11,117.010002,114.970001,114.089996,117.330002,1408600.0,WLTW


In [7]:
# Lets look for null values in the nyse securities dataset
nyse_sec_df.isna().sum() #198 na values in the date first added column

#drop Date first added and Address of Headquarters columns because we won't be needing them
nyse_sec_clean = nyse_sec_df.drop(["Date first added","Address of Headquarters"],axis = 1)
#Set index as Ticker symbol
#nyse_sec_clean = nyse_sec_clean.set_index(nyse_sec_clean["Ticker symbol"])
nyse_sec_clean.head()


,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,718877


In [8]:
# Lets look for null values in the SP500 dataset
spx_df.isna().sum() ## all good no null values

spx_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [9]:
# Lets merge the NYSE data sets together based on ticker symbol
combined_df = nyse_price_df.merge(nyse_sec_clean, on = "Ticker symbol",how ="inner")

combined_df = combined_df.set_index("Ticker symbol")

combined_df.head()

,date,open,close,low,high,volume,Security,SEC filings,GICS Sector,GICS Sub Industry,CIK
Ticker symbol,,,,,,,,,,,
WLTW,2016-01-05,123.430000,125.839996,122.309998,126.250000,2163600.0,Willis Towers Watson,reports,Financials,Insurance Brokers,1140536
WLTW,2016-01-06,125.239998,119.980003,119.940002,125.540001,2386400.0,Willis Towers Watson,reports,Financials,Insurance Brokers,1140536
WLTW,2016-01-07,116.379997,114.949997,114.930000,119.739998,2489500.0,Willis Towers Watson,reports,Financials,Insurance Brokers,1140536
WLTW,2016-01-08,115.480003,116.620003,113.500000,117.440002,2006300.0,Willis Towers Watson,reports,Financials,Insurance Brokers,1140536
WLTW,2016-01-11,117.010002,114.970001,114.089996,117.330002,1408600.0,Willis Towers Watson,reports,Financials,Insurance Brokers,1140536


# SP500 Analysis

In [103]:
#Filter from 1928-1960 for Visualization of the first section of years in this dataset.
spx_28_60_df["Date"] = pd.to_datetime(spx_28_60_df["Date"])
spx_28_60_df = spx_df[(spx_df["Date"]> "1928-01-01") & (spx_df["Date"]< "1960-01-01")] 
#Create Line plot of filtered Data

spx_28_60_df.hvplot.line(x='Date', y='Adj Close', title=' SP500 1928 -1960', value_label='Average Stock Closing Price',
                              frame_width=700, frame_height=500, group_label='Daily Average')


/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_15167/2532784904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_28_60_df["Date"] = pd.to_datetime(spx_28_60_df["Date"])


:Curve   [Date]   (Adj Close)

In [104]:
#Filter from 1960-1990 for Visualization of the first section of years in this dataset.
spx_60_90_df = spx_df[(spx_df["Date"]> "1960-01-01") & (spx_df["Date"]< "1990-01-01")] 
#Create Line plot of filtered Data
spx_60_90_df.hvplot.line(x='Date', y='Adj Close', title=' SP500 1960 -1990', value_label='Average Stock Closing Price',
                              frame_width=700, frame_height=500, group_label='Daily Average')

:Curve   [Date]   (Adj Close)

In [105]:
#Filter from 1990-2020 for Visualization of the first section of years in this dataset.
spx_90_2020_df = spx_df[(spx_df["Date"]> "1990-01-01") & (spx_df["Date"]< "2020-01-01")] 
#Create Line plot of filtered Data
spx_90_2020_df.hvplot.line(x='Date', y='Adj Close', title=' SP500 1990 -2020', value_label='Average Stock Closing Price',
                              frame_width=700, frame_height=500, group_label='Daily Average')

:Curve   [Date]   (Adj Close)

In [110]:
#Lets Create the Daily Change column
spx_df["Percent Change"] = (spx_df['Adj Close'] - spx_df['Open'])/spx_df['Open']*100
#Preview
spx_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Percent Change
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0


In [112]:
#Lets Describe the data to find out what Daily Change Values are considered average
spx_df["Percent Change"].describe() #mean = 0.018129, min = -20.466931, max = 10.789006

count    23323.000000
mean         0.018129
std          0.781987
min        -20.466931
25%         -0.121532
50%          0.000000
75%          0.212481
max         10.789006
Name: Percent Change, dtype: float64

In [118]:
#Lets filter the data down to only show dates when the daily change was above 10
spx_df.loc[spx_df["Percent Change"]>0,:] # There are 7802 values

#Lets go further to filter this range down to a smaller amount of data. Lets try 20 for Daily Change.
spx_df.loc[spx_df["Percent Change"]>3,:] # There are 97 values.

# Lets try to get the data to 10 rows or below to show about 10 of the biggest dates of Daily Change in the history of the SP500.
spx_loss_df = spx_df.loc[spx_df["Percent Change"]>5,:] # There are 11 values.
spx_loss_df


,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Percent Change
14994,1987-10-20,225.059998,245.619995,216.460007,236.830002,236.830002,608100000,11.770004,5.229718
14995,1987-10-21,236.830002,259.269989,236.830002,258.380005,258.380005,449600000,21.550003,9.099355
17529,1997-10-28,876.989990,923.090027,855.270020,921.849976,921.849976,1202550000,44.859986,5.115222
17745,1998-09-08,973.890015,1023.460022,973.890015,1023.460022,1023.460022,814800000,49.570007,5.089898
18331,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,64.290039,5.009861
18718,2002-07-24,797.710022,844.320007,775.679993,843.429993,843.429993,2775560000,45.719971,5.731402
18721,2002-07-29,852.840027,898.960022,852.840027,898.960022,898.960022,1778650000,46.119995,5.407813
20285,2008-10-13,912.750000,1006.929993,912.750000,1003.349976,1003.349976,7263370000,90.599976,9.926045
20296,2008-10-28,848.919983,940.510010,845.270020,940.510010,940.510010,7096950000,91.590027,10.789006
20308,2008-11-13,853.130005,913.010010,818.690002,911.289978,911.289978,7849120000,58.159973,6.817246


In [16]:
# The date ranges that appear here are 2008-10-13/10-28 Houseing Crash, 2018-02-06, 2018-12-26 # Not sure, 2020-03 Covid Pandemic

In [119]:
# Now I will look at the top dates per time period charted above. Here is 1928-1960
spx_28_60_df["Percent Change"] = spx_28_60_df["Close"] - spx_28_60_df["Open"]/spx_28_60_df["Open"]
spx_28_60_df["Percent Change"].max() # Daily Change Max was 0. Open and close must not have been calculated in this time peried.

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_15167/1787081445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_28_60_df["Percent Change"] = spx_28_60_df["Close"] - spx_28_60_df["Open"]/spx_28_60_df["Open"]


59.709999

In [131]:
#Here are the biggest Daily Changes from 1960-1990
spx_60_90_df["Percent Change"] = (spx_60_90_df["Close"] - spx_60_90_df["Open"])/spx_60_90_df["Open"]


spx_60_90_df.loc[spx_60_90_df["Percent Change"]>0.05,:]

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_15167/3581588425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_60_90_df["Percent Change"] = (spx_60_90_df["Close"] - spx_60_90_df["Open"])/spx_60_90_df["Open"]


,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Percent Change
14994,1987-10-20,225.059998,245.619995,216.460007,236.830002,236.830002,608100000,235.830002,0.052297
14995,1987-10-21,236.830002,259.269989,236.830002,258.380005,258.380005,449600000,257.380005,0.090994


In [137]:
#Here are the biggest Daily Changes from 1960-1990
spx_90_2020_df["Percent Change"] = (spx_90_2020_df["Close"] - spx_90_2020_df["Open"])/spx_90_2020_df["Open"]


spx_90_2020_df.loc[spx_90_2020_df["Percent Change"]>0.05,:] # Lots of values greater than 10, had to filter it to 70.

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_15167/996501554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_90_2020_df["Percent Change"] = (spx_90_2020_df["Close"] - spx_90_2020_df["Open"])/spx_90_2020_df["Open"]


,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Percent Change
17529,1997-10-28,876.989990,923.090027,855.270020,921.849976,921.849976,1202550000,44.859986,0.051152
17745,1998-09-08,973.890015,1023.460022,973.890015,1023.460022,1023.460022,814800000,49.570007,0.050899
18331,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,64.290039,0.050099
18718,2002-07-24,797.710022,844.320007,775.679993,843.429993,843.429993,2775560000,45.719971,0.057314
18721,2002-07-29,852.840027,898.960022,852.840027,898.960022,898.960022,1778650000,46.119995,0.054078
20285,2008-10-13,912.750000,1006.929993,912.750000,1003.349976,1003.349976,7263370000,90.599976,0.099260
20296,2008-10-28,848.919983,940.510010,845.270020,940.510010,940.510010,7096950000,91.590027,0.107890
20308,2008-11-13,853.130005,913.010010,818.690002,911.289978,911.289978,7849120000,58.159973,0.068172
20314,2008-11-21,755.840027,801.200012,741.020020,800.030029,800.030029,9495900000,44.190002,0.058465
20315,2008-11-24,801.200012,865.599976,801.200012,851.809998,851.809998,7879440000,50.609986,0.063168


In [141]:
# Because the events that showed up were very important events in global financial history, I want to actually filter the data down to inclide
#more nuanced changes in value, the kind that would happen more frequently throughout a fiscal year.

spx_90_2020_df.loc[spx_90_2020_df["Percent Change"].between(0.05,0.3),:]   

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Change,Percent Change
17529,1997-10-28,876.989990,923.090027,855.270020,921.849976,921.849976,1202550000,44.859986,0.051152
17745,1998-09-08,973.890015,1023.460022,973.890015,1023.460022,1023.460022,814800000,49.570007,0.050899
18331,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,64.290039,0.050099
18718,2002-07-24,797.710022,844.320007,775.679993,843.429993,843.429993,2775560000,45.719971,0.057314
18721,2002-07-29,852.840027,898.960022,852.840027,898.960022,898.960022,1778650000,46.119995,0.054078
20285,2008-10-13,912.750000,1006.929993,912.750000,1003.349976,1003.349976,7263370000,90.599976,0.099260
20296,2008-10-28,848.919983,940.510010,845.270020,940.510010,940.510010,7096950000,91.590027,0.107890
20308,2008-11-13,853.130005,913.010010,818.690002,911.289978,911.289978,7849120000,58.159973,0.068172
20314,2008-11-21,755.840027,801.200012,741.020020,800.030029,800.030029,9495900000,44.190002,0.058465
20315,2008-11-24,801.200012,865.599976,801.200012,851.809998,851.809998,7879440000,50.609986,0.063168


# Application Creation

In [142]:
# We would like our application to take a date input and output a list of headlines from the date selected as well as 5 
#entries before and after the date selected. 

# We will need to just limit and customize the output to what serves our purposes
# Save config information

# Base url
url = "https://api.nytimes.com/svc/archive/v1/"

# Here is where we will set the publish date as an input value
year = input("What year would you like to search: ")
month =input("What month would you like to search: ")


# Build query URL - change url to just look for pub_date
query_url = url + year + "/" + month + ".json?api-key=" + api_key
# Get NYT headline data
NYT_response = requests.get(query_url).json()


# Get the temperature from the response

# convert the date time format into the same format as the combined_df by getting rid of the last 14 digits.


KeyboardInterrupt: Interrupted by user

In [75]:
#initialize column lists for NYT_df
headline=[]
news_desk = []
section = []
date = []
# print(NYT_response['response']['docs'][1])
for i in range(len(NYT_response['response']['docs'])):
    headline.append(NYT_response['response']['docs'][i]['headline']['print_headline'])
    news_desk.append(NYT_response['response']['docs'][i]['news_desk'])
    section.append(NYT_response['response']['docs'][i]['section_name'])
    date.append(NYT_response['response']['docs'][i]['pub_date'])

# print(Headline)
# len(NYT_response['response']['docs'])

In [85]:
#create NYT dataframe
NYT_dict = {
    "headline": headline,
    "News Desk": news_desk,
    "Section": section,
    "Date": date
}
NYT_df = pd.DataFrame(NYT_dict)
NYT_df[["Date","Time"]] = NYT_df.Date.str.split("T",expand = True)
# input pub date
day = input("What day would you like to search: ")
#BUild pub_date string
pub_date = f'{year}-{month}-{day}'
pub_date = datetime.strptime(pub_date,'%Y-%m-%d')
pub_date = (pub_date) - timedelta(days = 3)
#NYT_df["Day"] = NYT_df["Date"].str.slice(start = 8)
NYT_df = NYT_df.drop(["Time"], axis = 1) 
#filter data by pub_date


What day would you like to search: 01


In [94]:
NYT_df["Date"] = pd.to_datetime(NYT_df["Date"])
#NYT_df["Date Delta"]=NYT_df["Date"]
NYT_df_new = NYT_df.loc[NYT_df["Date"]=='2016-03-01']

2016-02-27


,headline,News Desk,Section,Date
0,,Politics,U.S.,2016-03-01
1,"A Nightclub Era Relived, Via Gershwin and More",Culture,Arts,2016-03-01
2,"Gawker Staff, Now Union, Negotiates First Cont...",Business,Business Day,2016-03-01
3,Autonomous Google Car Hits Public Bus During T...,Business,Technology,2016-03-01
4,Justices Look at Recusals in Capital Trials,National,U.S.,2016-03-01
...,...,...,...,...
237,Paul L. Whalen,Business,Real Estate,2016-03-01
238,,,Blogs,2016-03-01
239,Transactions,Business,Real Estate,2016-03-01
240,South Dakota Governor Vetoes Bathroom Limit on...,National,U.S.,2016-03-01


In [95]:
NYT_df_new["headline"].replace("",np.nan,inplace = True)
NYT_df_new.dropna(inplace= True)
NYT_df_new.loc[NYT_df_new["News Desk"]== "Business",:]

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_15167/46686881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYT_df_new["headline"].replace("",np.nan,inplace = True)
/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_15167/46686881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYT_df_new.dropna(inplace= True)


,headline,News Desk,Section,Date
2,"Gawker Staff, Now Union, Negotiates First Cont...",Business,Business Day,2016-03-01
3,Autonomous Google Car Hits Public Bus During T...,Business,Technology,2016-03-01
8,A Paper Basks in ‘Spotlight’ Oscar,Business,Business Day,2016-03-01
10,Investigation Is Urged of Lobbyist for Lenders,Business,Business Day,2016-03-01
14,Discontent in Britain Echoes Rise of Trump,Business,Business Day,2016-03-01
44,Researchers Describe Costly Waste in Cancer Drugs,Business,Health,2016-03-01
64,Glencore Loss Reflects Hard Times in Mining,Business,Business Day,2016-03-01
65,"Barclays, Posting Loss, Plans to Reduce Busin...",Business,Business Day,2016-03-01
81,Owner of N.Y.S.E. Confirms Interest in Buying ...,Business,Business Day,2016-03-01
107,"Honeywell, Facing Resistance on Antitrust Grou...",Business,Business Day,2016-03-01
